# Connect to G drive

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# !zip -r '/content/drive/MyDrive/NLP/ATE_models.zip' '/content/drive/MyDrive/NLP/Final/Models'

In [4]:
# !zip -r '/content/drive/MyDrive/NLP/APC_models.zip' '/content/drive/MyDrive/NLP/Akanksha_files/'

In [5]:
import pickle
import joblib

#Load Save Pickle Utility
def save(filename, obj):
  with open(filename, 'wb') as handle:
      joblib.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load(filename):
  with open(filename, 'rb') as handle:
      return joblib.load(filename)


# Import 

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd 
import os
import pickle
import numpy as np
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset

import os
import torch
import torch.nn as nn
import argparse
from sklearn import metrics

In [7]:
from torch.utils.data import DataLoader

# Models 

In [8]:
class Squeezer(nn.Module):
    
    #Squeeze sequence embedding length to the longest one in the batch
    
    def __init__(self, batch_first=True):
        super(Squeezer, self).__init__()
        self.batch_first = batch_first
    
    def forward(self, x, x_len):
        # sequence , sort , pad and pack , unpack , unsort
        
        # sorting the given x lengths values 
        x_sort_idx = torch.sort(x_len, descending=True)[1].long()
        x_unsort_idx = torch.sort(x_sort_idx)[1].long()
        x_len = x_len[x_sort_idx]
        x_len = np.array(x_len.cpu())
        x = x[x_sort_idx]
        '''pack'''
        x_emb_p = torch.nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=self.batch_first)
        '''unpack'''
        out, _ = torch.nn.utils.rnn.pad_packed_sequence(x_emb_p, batch_first=self.batch_first)
        if self.batch_first:
            out = out[x_unsort_idx]
        else:
            out = out[:, x_unsort_idx]
        return out

In [9]:
class Recurrent_Models(nn.Module):
    '''
    LSTM which can hold variable length sequence, use like TensorFlow's RNN(input, lenght...).
    '''
    def __init__(self, input_size, hidden_size, num_layers=1, bias=True, batch_first=True, dropout=0,
                 bidirectional=False, only_use_last_hidden_state=False, rnn_type='LSTM'):
        super(Recurrent_Models, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bias = bias
        self.batch_first = batch_first
        self.dropout = dropout
        self.bidirectional = bidirectional
        self.only_use_last_hidden_state = only_use_last_hidden_state
        self.rnn_type = rnn_type
        
        if self.rnn_type == 'LSTM':
            self.RNN = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                               bias=bias, batch_first=batch_first, dropout=dropout, bidirectional=bidirectional)
        elif self.rnn_type == 'GRU':
            self.RNN = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                              bias=bias, batch_first=batch_first, dropout=dropout, bidirectional=bidirectional)
        elif self.rnn_type == 'RNN':
            self.RNN = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers,
                              bias=bias, batch_first=batch_first, dropout=dropout, bidirectional=bidirectional)
    
    def forward(self, x, x_len):
        
        '''
        sequence -> sort -> pad and pack -> process using RNN -> unpack -> unsort
        '''
        '''sort'''
        x_sort_idx = torch.sort(x_len, descending=True)[1].long()
        x_unsort_idx = torch.sort(x_sort_idx)[1].long()

        
        x_len = x_len[x_sort_idx]
        x_len = np.array(x_len.cpu())
        x = x[x_sort_idx]
        
        '''pack'''
        x_emb_p = torch.nn.utils.rnn.pack_padded_sequence(x, x_len, batch_first=self.batch_first)
        ''' process '''
        if self.rnn_type == 'LSTM':
            out_pack, (ht, ct) = self.RNN(x_emb_p, None)
        else:
            out_pack, ht = self.RNN(x_emb_p, None)
            ct = None
        '''unsort'''
        ht = ht[:, x_unsort_idx]
        if self.only_use_last_hidden_state:
            return ht
        else:
            out, _ = torch.nn.utils.rnn.pad_packed_sequence(out_pack, batch_first=self.batch_first)
            if self.batch_first:
                out = out[x_unsort_idx]
            else:
                out = out[:, x_unsort_idx]
            if self.rnn_type == 'LSTM':
                ct = ct[:, x_unsort_idx]
            return out, (ht, ct)

### PBAN

In [10]:
# PBAN Model 
class PBAN(nn.Module):
    # Position-aware bidirectional attention network 
    # paper https://aclanthology.org/C18-1066.pdf

    # if embedding matrix not one then glove or word to vec used else brt used for embeddings 
    # hyper_param is a dictionary containing all required hyperparameters 
    def __init__(self, embedding_matrix, hyper_param):

        super(PBAN, self).__init__()
        # embeddings of the text 
        
        self.embed = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))
   
     
        # randomly intialised embedding for position with respect to aspect  
        self.pos_embed = nn.Embedding(hyper_param['max_length'], hyper_param['position_dim'])
        # Left GRU taking aspect 
        self.aspect_gru = Recurrent_Models(hyper_param['embed_dim'], hyper_param['hidden_dim'], num_layers=1, 
                                    batch_first=True, bidirectional=True, rnn_type='GRU')
        # Right GRU taking input as words and position embedding 
        self.sentence_position_gru = Recurrent_Models(hyper_param['embed_dim']+hyper_param['position_dim'], hyper_param['hidden_dim'], num_layers=1, 
                                     batch_first=True, bidirectional=True, rnn_type='GRU')
        

        self.weight_m = nn.Parameter(torch.Tensor(hyper_param['hidden_dim']*2, hyper_param['hidden_dim']*2))
        self.bias_m = nn.Parameter(torch.Tensor(1))
        self.weight_n = nn.Parameter(torch.Tensor( hyper_param['hidden_dim']*2, hyper_param['hidden_dim']*2))
        self.bias_n = nn.Parameter(torch.Tensor(1))
        self.w_r = nn.Linear( hyper_param['hidden_dim']*2,  hyper_param['hidden_dim'])
        self.w_s = nn.Linear( hyper_param['hidden_dim'], hyper_param['polarities_dim'])
    
    def forward(self, X):
        # getting text , aspect term and position indices 
        text, aspect_text, position_tag = X[0], X[1], X[2]
  
        x = self.embed(text)
          #Sentence representation from any glove or wrod to vec '''
          
        ''' Hidden Representation from Sentence and Position knowledge '''
        # getting position embedding 
        position_embedding = self.pos_embed(position_tag)
        # getting lengths of all batch sentences into a array x_len (removing padded 0)
        x_len = torch.sum(text != 0, dim=-1)
        # passing sentence and position embedding to gru 
        x = torch.cat((position_embedding, x), dim=-1)
        h_x, _ = self.sentence_position_gru(x, x_len)

        ''' Aspect term representation '''
        aspect_embedding = self.embed(aspect_text)
        aspect_len = torch.sum(aspect_text != 0, dim=-1)
        h_t, _ = self.aspect_gru(aspect_embedding, aspect_len)

        # Attention Calculation 
        ''' Aspect term to position-aware sentence attention '''
         
        alpha = F.softmax(torch.tanh(torch.add(torch.bmm(torch.matmul(h_t, self.weight_m), torch.transpose(h_x, 1, 2)), self.bias_m)), dim=1)
        s_x = torch.bmm(alpha, h_x)
        ''' Position-aware sentence attention to aspect term '''
        h_x_pool = torch.unsqueeze(torch.div(torch.sum(h_x, dim=1), x_len.float().view(x_len.size(0), 1)), dim=1)
        gamma = F.softmax(torch.tanh(torch.add(torch.bmm(torch.matmul(h_x_pool, self.weight_n), torch.transpose(h_t, 1, 2)), self.bias_n)), dim=1)
        h_r = torch.squeeze(torch.bmm(gamma, s_x), dim=1)
        ''' Output transform '''
        out = torch.tanh(self.w_r(h_r))
        out = self.w_s(out)
        return out

### LSTM

In [11]:
class LSTM(nn.Module):
    # standard LSTM output without any consideration of target  
    def __init__(self, embedding_matrix, hyper_param):
        super(LSTM, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))
        self.lstm = Recurrent_Models(hyper_param['embed_dim'], hyper_param['hidden_dim'], num_layers=1, batch_first=True)
        self.out = nn.Linear(hyper_param['hidden_dim'], hyper_param['polarities_dim'])
    
    def forward(self, x):
        # extracting only text 
        
        text = x[0]
        # substituting words by embedding
        
        x = self.embedding(text)
        x_len = torch.sum(text != 0, dim=-1)
        
        _, (hidden, _) = self.lstm(x, x_len)
        out = self.out(hidden[0])
        return out

### AE-LSTM 

In [12]:
class AE_LSTM(nn.Module):
    ''' LSTM with Aspect Embedding '''
    def __init__(self, embedding_matrix, hyper_param):
        super(AE_LSTM, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(torch.tensor(embedding_matrix, dtype=torch.float))
        self.squeeze_embedding = Squeezer()
        self.lstm = Recurrent_Models(hyper_param['embed_dim']*2, hyper_param['hidden_dim'], num_layers=1, batch_first=True)
        self.dense_layer = nn.Linear(hyper_param['hidden_dim'], hyper_param['polarities_dim'])
    
    def forward(self, x):
        # extracting text and aspect words from given batch of sentences
        text_words, aspect_text = x[0], x[1]
        
        # finding max length among all aspects and text  
        x_len = torch.sum(text_words != 0, dim=-1)
        x_len_max = torch.max(x_len)
        aspect_len = torch.sum(aspect_text != 0, dim=-1).float()
        
        # Geting embedding of aspects and words or texts 
        x = self.embedding(text_words)
        x = self.squeeze_embedding(x, x_len)
        aspect = self.embedding(aspect_text)
        # concatenating aspect and all the words respectively 
        aspect_pool = torch.div(torch.sum(aspect, dim=1), aspect_len.view(aspect_len.size(0), 1))
        aspect = torch.unsqueeze(aspect_pool, dim=1).expand(-1, x_len_max, -1)
        x = torch.cat((aspect, x), dim=-1)
        # sending input to lstm 
        _, (hidden, _) = self.lstm(x, x_len)
        out = self.dense_layer(hidden[0])
        return out

# Useful Functions 

### Training 

In [13]:
def train( test_dataloader,model,train_dataloader, epochs, criterion, optimizer, save_path, max_test_acc_overall=0):
  
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    max_test_acc = 0
    max_f1 = 0
    global_step = 0
    best_path = None
    index_no = 1

    for idx,epoch in enumerate(range(epochs)):
        print("--------------")
        print('epoch:', epoch)

        correct, total = 0, 0
        for i_batch, batch in enumerate(train_dataloader):
            # training model in training mode in each epoch 
            global_step += 1
            model.train()
            optimizer.zero_grad()
            
            inputs = [batch[col].to(device) for col in ['text', 'aspect', 'position']]
            outputs = model(inputs)
            targets = batch['polarity'].to(device)
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            if global_step % 5 == 0:
                # finding training accuracy 
                correct += (torch.argmax(outputs, -1) == targets).sum().item()
                total += len(outputs)
                train_acc = correct / total
                # finding testing accuracy and evaluation 
                test_acc, f1 = evaluate(model,test_dataloader)
                if test_acc > max_test_acc:
                    max_test_acc = test_acc
                    
                if f1 > max_f1:
                    max_f1 = f1
                    if not os.path.exists(save_path + 'state_dict'):
                        os.mkdir(save_path + 'state_dict')
                    path = save_path + 'state_dict/{0}_{1}_{2}f1{3:.4f}_'.format('lstm', 'res', str(3) , f1) + str(index_no)
                    index_no = index_no+1 
                    torch.save(model.state_dict(), path)
                    print('model saved:', path)

                    best_path = path
                print('loss: {:.4f}, acc: {:.4f}, test_acc: {:.4f}, f1: {:.4f}'.format(loss.item(), train_acc, test_acc, f1))
    return max_test_acc, max_f1, best_path

### Evaluation 

In [14]:
def evaluate(model,test_dataloader):
    # switch model to evaluation mode
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    test_correct, test_total = 0, 0
    y_complete, y_hat_complete = None, None
    
    with torch.no_grad():
        for idx,batch in enumerate(test_dataloader):
            
            x = [batch[col].to(device) for col in ['text', 'aspect', 'position']]
            y = batch['polarity'].to(device)
            outputs = model(x)
            
            test_correct += (torch.argmax(outputs, -1) == y).sum().item()
            test_total += len(outputs)
            # appending all y and y hats  
            if y_complete is not None:
              y_complete = torch.cat((y_complete, y), dim=0)
            else:
              y_complete = y
            if y_hat_complete is not None:
              y_hat_complete = torch.cat((y_hat_complete, outputs), dim=0)
            else:
              y_hat_complete = outputs

    test_acc = test_correct / test_total
    f1 = metrics.f1_score(y_complete.cpu(), torch.argmax(y_hat_complete, -1).cpu(), labels=[0, 1, 2], average='macro')
    return test_acc, f1


### resetting hyperparameters

In [15]:
def reset_parameters(model):
  for m in model.parameters():
      if m.requires_grad:
          if len(m.shape) > 1:
              torch.nn.init.xavier_uniform_(m)
          else:
              stdv = 1. / (m.shape[0]**0.5)
              torch.nn.init.uniform_(m, a=-stdv, b=stdv)

In [16]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

device(type='cuda')

# Data Preprocessing 

In [17]:
def getResults(data):
  rows = []
  # Iterate through every row
  for i in range(len(data.index)):
      # Collect all occurences of B
      indices_B = [k for k, x in enumerate(data.loc[i]["aspect_labels"]) if x == 'B']

      tempListA = data.loc[i]["terms"]

      # Iterate through every index
      for j in indices_B:
        tempListB = ['O'] * len(data.loc[i][0])
        tempListB[j] = "B"

        for k in range(j + 1, len(data.loc[i][0])):
          if data.loc[i]["aspect_labels"][k] == "I":
            if tempListB[k-1] == "B" or tempListB[k-1] == "I":
              tempListB[k] = "I"

        tempListC = ['NA'] * len(data.loc[i][0])
        senti = data.loc[i]["sentiments"][j]
      
      
        rows.append([tempListA, tempListB, senti])

  result = pd.DataFrame(rows, columns = ["terms", "aspect_labels", "sentiments"])

  return result

In [18]:
def pad_sentence(arr,val,max_len):
  
  times = max_len - len(arr)

  if val == 't':
    temp = [0]*times
    return arr+temp

  if val == 'a':
    temp = ['O']*times
    return arr+temp

  if val == 's':
    temp = ['NA']*times
    return arr+temp


In [19]:
def create_vocab(arr):
  vocab = dict()
  idx = 0 
  # padding represented by 0 and on index 0 
  vocab[0] = 0
  idx = idx+1
  vocab['unk'] = 1
  
  for sen in arr:
    for w in sen:
      if w not in vocab.keys() and  w != 0:
        idx = idx+1
        vocab[w] = idx
  inv_vocab = {v: k for k, v in vocab.items()} 
  return vocab , inv_vocab

def word_to_id(vocab,w):
  if w not in vocab.keys():
    return 1
  else:
    return vocab[w]


def id_to_word(inv_vocab,id):
  return inv_vocab[id]
 


In [20]:
def prepare_dataset(vocab, df, max_length):

  number = df.shape[0]
  # returns all text element in data 
  text_data = np.zeros([number,max_length])
  aspect_data = np.zeros([number,max_length])
  position_data = np.zeros([number,max_length])
  sentiment = np.zeros([number])

  # flag to keep a track of how many aspect terms inserted 
  
  for idx, sen in enumerate(df['terms'].to_numpy()):
    flag = 0
    for wid , w in enumerate(sen):
      # text number pusshed 
      text_data[idx][wid] = word_to_id(vocab,w)

      # aspect number pushed 
      if df['aspect_labels'].iloc[idx][wid] == 'B' or df['aspect_labels'].iloc[idx][wid] == 'I':
        
        aspect_data[idx][flag] = word_to_id(vocab,w)
        flag = flag + 1
        
    # sentiment of sentence added 
    if df['sentiments'][idx] == 'NEU':
      sentiment[idx] = 2
    elif df['sentiments'][idx]  == 'POS':
      sentiment[idx] = 0 
    else:
      sentiment[idx] = 1

    # position embedding 
    x1 = np.array(df['aspect_labels'][idx])
    start = np.where(x1 == 'B')[0][0]
    if not len(np.where(x1 == 'I')[0]):
      end = start+1
    else:
      end = np.where(x1 == 'I')[0][-1]
      end = end+1
    # print(x1)
    # print("-----")
    # print(start,end)

    x = df['terms'][idx]
    text_left = list(x[:start])
    text_aspect = list(x[start:end])
    text_right = list(x[end:])
    tag_left = [len(text_left)-i for i in range(len(text_left))]
    tag_aspect = [0 for i in range(len(text_aspect))]
    tag_right = [i+1 for i in range(len(text_right))]
    position_tag = tag_left + tag_aspect + tag_right
    if len(position_tag) == 0:
        position_tag = [0]

    times = max_length - len(position_tag)
    r = [0]*times
    position_tag = position_tag+r
    position_data[idx] = np.array(position_tag)

  return text_data , aspect_data, position_data, sentiment 
      


In [21]:
def prepare_text_file_train(path,max_length):
  # max_length = 80
  data = load(path)
  df = pd.DataFrame(columns = ['terms', 'aspect_labels', 'sentiments'])

  text = []
  aspect = []
  sent = []

  for sample in data:
    aspect.append(sample['labels'] )
    text.append(sample['terms'] )
    sent.append(sample['sentiment'] )
  
  df['terms'] = text 
  df['aspect_labels'] = aspect
  df['sentiments'] = sent

  my_df = getResults(df) 
  v , vr = create_vocab(my_df['terms'].to_numpy())
  text_data , aspect_data, position_data, sentiment  = prepare_dataset(v, my_df, max_length)

  return v, text_data , aspect_data, position_data, sentiment

def prepare_text_file_test(path,max_length,v):
  # max_length = 80
  data = load(path)
  df = pd.DataFrame(columns = ['terms', 'aspect_labels', 'sentiments'])

  text = []
  aspect = []
  sent = []

  for sample in data:
    aspect.append(sample['labels'] )
    text.append(sample['terms'] )
    sent.append(sample['sentiment'] )
  
  df['terms'] = text 
  df['aspect_labels'] = aspect
  df['sentiments'] = sent

  my_df = getResults(df) 

  text_data , aspect_data, position_data, sentiment  = prepare_dataset(v, my_df, max_length)

  return text_data , aspect_data, position_data, sentiment


In [22]:
# max_length = 80
# # t_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/train.pkl'
# # test_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/test.pkl'

# t_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/train.pkl'
# test_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/test.pkl'

# vocab, train_text_data , train_aspect_data, train_position_data, train_sentiment = prepare_text_file_train(t_path,max_length)
# test_text_data , test_aspect_data, test_position_data, test_sentiment = prepare_text_file_test(test_path,max_length,vocab)

In [23]:
# embedding_vector = {}
# f = open('/content/drive/MyDrive/NLP/glove.6B.300d.txt')
# for line in f:
#     value = line.split(' ')
#     word = value[0]
#     coef = np.array(value[1:],dtype = 'float32')
#     embedding_vector[word] = coef

In [24]:
# # Embedding from glove 

# embedding_matrix = np.zeros((len(vocab.keys()),300))
# for word,i in vocab.items():
#     embedding_value = embedding_vector.get(word)
#     if embedding_value is not None:
#         embedding_matrix[i] = embedding_value


In [25]:
# print(len(vocab.keys())) 

In [26]:
# embedding_matrix.shape

In [27]:
# base = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/'
# save(base + 'vocab.pkl',vocab)
# save(base + 'embedding_matrix.pkl',embedding_matrix)

# base = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/'
# save(base + 'vocab.pkl',vocab)
# save(base + 'embedding_matrix.pkl',embedding_matrix)

# Data Loader

In [28]:
class SentenceDataset(Dataset):
    
    def __init__(self, text_data , aspect_data, position_data, sentiment):
        data = list()
       
        for idx , text in enumerate(text_data):
          aspect_term = aspect_data[idx]
          position = position_data[idx]
          polarity = sentiment[idx]
          data.append({'text': torch.tensor(text,dtype=int), 'aspect': torch.tensor(aspect_term,dtype=int), 'position': torch.tensor(position,dtype=int), 'polarity':torch.tensor(polarity,dtype=int) })

        self._data = data
    
    def __getitem__(self, index):
        return self._data[index]
    
    def __len__(self):
        return len(self._data)


In [ ]:
# d = next(iter(train_dataloader))

In [ ]:
# d['text'][0]

tensor([ 656, 1057,  809,  588,    2,  171,   34,   87,  426,   12,   49,  345,
         609,   38,  595,   38,    7, 1480,   19,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])

In [ ]:
# d['aspect'][0]

tensor([595,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [ ]:
# d['position'][0]

tensor([14, 13, 12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1,  0,  1,  2,  3,
         4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0])

In [ ]:
# d['polarity'][0]

tensor(2)

# Model and Training

In [29]:
# initializers = {
#     'xavier_uniform_': torch.nn.init.xavier_uniform_,
#     'xavier_normal_': torch.nn.init.xavier_normal_,
#     'orthogonal_': torch.nn.init.orthogonal_,
# }

# optimizers = {
#     'adadelta': torch.optim.Adadelta,  # default lr=1.0
#     'adagrad': torch.optim.Adagrad,    # default lr=0.01
#     'adam': torch.optim.Adam,          # default lr=0.001
#     'adamax': torch.optim.Adamax,      # default lr=0.002
#     'asgd': torch.optim.ASGD,          # default lr=0.01
#     'rmsprop': torch.optim.RMSprop,    # default lr=0.01
#     'sgd': torch.optim.SGD,
# }

In [40]:

params = { 
'lstm': {
'data' :  ['text'],
'hyperparameters':  {'max_length' : 80,
'position_dim' : 100,
'embed_dim' : 300,
'hidden_dim' : 5,
'polarities_dim' : 3,
'epochs': 20 , 
'initializer': torch.nn.init.xavier_uniform_ , 
'optimizer':torch.optim.Adam, 
'lr': 1e-4 , 

  }  , 

},
'aelstm' : {
        'data' :  ['text', 'aspect'],
'hyperparameters': {'max_length' : 80,
'position_dim' : 100,
'embed_dim' : 300,
'hidden_dim' : 200,
'polarities_dim' : 3,
'epochs': 20 , 
'initializer': torch.nn.init.xavier_uniform_ , 
'optimizer': torch.optim.Adam, 
'lr': 1e-3 , 

  }  , 

},
'pban' : {
        'data' : ['text', 'aspect', 'position'] ,
'hyperparameters': {'max_length' : 80,
'position_dim' : 100,
'embed_dim' : 300,
'hidden_dim' : 200,
'polarities_dim' : 3,
  'epochs': 20 , 
'initializer': torch.nn.init.xavier_uniform_ , 
'optimizer':torch.optim.Adam, 
'lr': 1e-3 , 

  }  
}
}


In [41]:

max_length = 80

x = input("Name the model you want to input 'lstm, aelstm, pban : ")
model_name = x
model_data = params[model_name]['data'] 
model_hyper_param = params[model_name]['hyperparameters'] 

x = input("Enter dataset to be used 1. Restraunt 2.Laptop ")
if int(x) ==1:
  t_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/train.pkl'
  test_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/test.pkl'
  vocab = load('/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/vocab.pkl')
  embedding_matrix = load('/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/embedding_matrix.pkl')

else:
  t_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/train.pkl'
  test_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/test.pkl'
  vocab = load('/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/vocab.pkl')
  embedding_matrix = load('/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/embedding_matrix.pkl')


if model_name == 'lstm' and int(x) == 1:
  save_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/models/lstm/'
elif model_name == 'aelstm' and int(x) == 1:
  save_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/models/aelstm/'
elif model_name == 'pban' and int(x) == 1:
  save_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/models/pban/'
elif model_name == 'lstm' and int(x) == 2:
  save_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/lstm/'
elif model_name == 'aelstm' and int(x) == 2:
  save_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/aelstm/'
elif model_name == 'pban' and int(x) == 2:
  save_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/pban/'



Name the model you want to input 'lstm, aelstm, pban : pban
Enter dataset to be used 1. Restraunt 2.Laptop 1


In [32]:
# !rm -r '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/lstm/'

In [42]:

vocab, train_text_data , train_aspect_data, train_position_data, train_sentiment = prepare_text_file_train(t_path,max_length)
test_text_data , test_aspect_data, test_position_data, test_sentiment = prepare_text_file_test(test_path,max_length,vocab)

trainset = SentenceDataset(train_text_data , train_aspect_data, train_position_data, train_sentiment)
testset = SentenceDataset( test_text_data , test_aspect_data, test_position_data, test_sentiment)

train_dataloader = DataLoader(dataset=trainset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset=testset, batch_size=64, shuffle=False)

In [43]:
model_name , model_data , model_hyper_param 

('pban',
 ['text', 'aspect', 'position'],
 {'embed_dim': 300,
  'epochs': 20,
  'hidden_dim': 200,
  'initializer': <function torch.nn.init.xavier_uniform_>,
  'lr': 0.001,
  'max_length': 80,
  'optimizer': torch.optim.adam.Adam,
  'polarities_dim': 3,
  'position_dim': 100})

In [44]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
if model_name == 'lstm':
  model = LSTM(embedding_matrix,model_hyper_param).to(device)
elif model_name == 'aelstm':
  model = AE_LSTM(embedding_matrix,model_hyper_param).to(device)
else:
  model = PBAN(embedding_matrix,model_hyper_param).to(device)


criterion = nn.CrossEntropyLoss()
_params = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(_params, lr=model_hyper_param['lr'], weight_decay=1e-5)


# Training

In [ ]:
max_test_acc_overall = 0
max_f1_overall = 0

for i in range(1):
  print('repeat:', i)
  reset_parameters(model)
  max_test_acc, max_f1, best_path = train( test_dataloader,model,train_dataloader, model_hyper_param['epochs'],criterion, optimizer,save_path, max_test_acc_overall)
  print('max_test_acc: {0}, max_f1: {1}'.format(max_test_acc, max_f1))
  max_test_acc_overall = max(max_test_acc, max_test_acc_overall)
  max_f1_overall = max(max_f1, max_f1_overall)


print('max_test_acc_overall:', max_test_acc_overall)
print('max_f1_overall:', max_f1_overall)

repeat: 0
--------------
epoch: 0
model saved: /content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/lstm/state_dict/lstm_res_3f10.1377_1
loss: 1.3350, acc: 0.2344, test_acc: 0.2603, f1: 0.1377
loss: 1.2987, acc: 0.2266, test_acc: 0.2603, f1: 0.1377
loss: 1.2964, acc: 0.2344, test_acc: 0.2603, f1: 0.1377
loss: 1.2962, acc: 0.2227, test_acc: 0.2603, f1: 0.1377
loss: 1.2621, acc: 0.2219, test_acc: 0.2603, f1: 0.1377
loss: 1.2761, acc: 0.2214, test_acc: 0.2603, f1: 0.1377
--------------
epoch: 1
loss: 1.3593, acc: 0.1094, test_acc: 0.2603, f1: 0.1377
loss: 1.2532, acc: 0.1484, test_acc: 0.2603, f1: 0.1377
loss: 1.2883, acc: 0.1510, test_acc: 0.2603, f1: 0.1377
loss: 1.2505, acc: 0.1641, test_acc: 0.2603, f1: 0.1377
loss: 1.2654, acc: 0.1625, test_acc: 0.2603, f1: 0.1377
loss: 1.2253, acc: 0.1693, test_acc: 0.2603, f1: 0.1377
--------------
epoch: 2
loss: 1.2310, acc: 0.2188, test_acc: 0.2603, f1: 0.1377
loss: 1.2399, acc: 0.2031, test_acc: 0.2603, f1: 0.1377
loss: 1.2079,

# Evaluate Trained Model 

In [36]:
  # path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/pban/state_dict/lstm_res_3f10.6111_9'
  # # model 
  # model = PBAN(embedding_matrix,model_hyper_param).to(device)
  # # load and evaluate 
  # model.load_state_dict(torch.load(path))
  # model.eval()
  # test_acc, f1 = evaluate(model,test_dataloader)
  # test_acc, f1

In [45]:
def replicate_results(path, model_name,test_dataloader,model_hyper_param):
# path of saved model 
  
  # model 
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
  if model_name == 'lstm':
    model = LSTM(embedding_matrix,model_hyper_param).to(device)
  elif model_name == 'aelstm':
    model = AE_LSTM(embedding_matrix,model_hyper_param).to(device)
  else:
    model = PBAN(embedding_matrix,model_hyper_param).to(device)

  # load and evaluate 
  model.load_state_dict(torch.load(path))
  model.eval()
  test_acc, f1 = evaluate(model,test_dataloader)
  print("Accuracy , F1 score")
  print(test_acc, f1)

In [46]:
model_name = input("Enter the model name : lstm, aelstm, pban : ")
x = input("Enter dataset : 1.Restraunt 2.Laptop")

# best_path with respect to each model and dataset
# 1 is rest , 2 is laptop 
if model_name == 'lstm' and int(x) == 1:
  best_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/models/lstm/state_dict/lstm_res_3f10.3224_19'
elif model_name == 'aelstm' and int(x) == 1:
  best_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/models/aelstm/state_dict/lstm_res_3f10.5655_12'
elif model_name == 'pban' and int(x) == 1:
  best_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_restraunt/models/pban/state_dict/lstm_res_3f10.6684_17'
elif model_name == 'lstm' and int(x) == 2:
  best_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/lstm/state_dict/lstm_res_3f10.3946_19'
elif model_name == 'aelstm' and int(x) == 2:
  best_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/aelstm/state_dict/lstm_res_3f10.5260_10'
elif model_name == 'pban' and int(x) == 2:
  best_path = '/content/drive/MyDrive/NLP/Akanksha_files/data_pkl_files_laptop/models/pban/state_dict/lstm_res_3f10.6111_9'

path = best_path 
print("Testing")
replicate_results(path, model_name,test_dataloader,model_hyper_param)
print("Training")
replicate_results(path, model_name,train_dataloader,model_hyper_param)

Enter the model name : lstm, aelstm, pban : pban
Enter dataset : 1.Restraunt 2.Laptop1
Testing
Accuracy , F1 score
0.7698483496877788 0.6684332275860755
Training
Accuracy , F1 score
0.7892321810951362 0.7088760125337403


### Restraunt 

In [ ]:
# Enter the model name : lstm, aelstm, pban : lstm
# Enter dataset : 1.Restraunt 2.Laptop1
# Testing
# Accuracy , F1 score
# 0.655664585191793 0.32236728458044067
# Training
# Accuracy , F1 score
# 0.6191495870296727 0.3743628482504735

In [ ]:
# Enter the model name : lstm, aelstm, pban : aelstm
# Enter dataset : 1.Restraunt 2.Laptop1
# Testing
# Accuracy , F1 score
# 0.6913470115967886 0.5655422938412333
# Training
# Accuracy , F1 score
# 0.7791373508718262 0.727080162045529

In [ ]:
# Testing
# Accuracy , F1 score
# 0.7698483496877788 0.6684332275860755
# Training
# Accuracy , F1 score
# 0.7892321810951362 0.7088760125337403

### Laptop

In [ ]:
# Enter the model name : lstm, aelstm, pban : lstm
# Enter dataset : 1.Restraunt 2.Laptop2
# Testing
# Accuracy , F1 score
# 0.48264984227129337 0.3945839746911814
# Training
# Accuracy , F1 score
# 0.4603718199608611 0.3988914190229198

In [ ]:
# Testing
# Accuracy , F1 score
# 0.5977917981072555 0.5259933986993969
# Training
# Accuracy , F1 score
# 0.8023483365949119 0.7789840806481317

In [ ]:
# Enter the model name : lstm, aelstm, pban : pban
# Enter dataset : 1.Restraunt 2.Laptop2
# Testing
# Accuracy , F1 score
# 0.6782334384858044 0.6110862356052444
# Training
# Accuracy , F1 score
# 0.850293542074364 0.8330121117125563